## keywords parsing

In [25]:
import re
import mmap
from pprint import pprint
import json
from pprint import pprint

import pandas as pd

In [26]:
import mmap

class Keyword():
    def __init__(self, kw): self.__dict__.update(kw)
    def get_name(self):     return self.ID if 'ID' in self.__dict__ else self.IC
    def get_category(self): return self.CA if 'ID' in self.__dict__ else self.IC
    def is_category(self):  return 'IC' in self.__dict__
    def is_keyword(self):   return 'ID' in self.__dict__
    
    def __getattr__(self, attr): return self.__dict__.get(attr, None)
    def __repr__(self): return repr(self.__dict__)
    def __str__(self): return repr(self)
    

def parse_keywlist(path):

    id_re = re.compile(rb'(ID|IC) \s+ ([^.]+) .\s+  AC \s+ (KW-\d{4}) \s+', re.X)
    multiline_re = re.compile(rb'//\s+|(\w\w)\s+([^\.]+)\.\s+', re.S)
    kw_split_re = re.compile('\s*[;:]\s*')

    kw_names = {}
    kw_acs = {}

    with open(path, 'rb+') as file:
        mem = mmap.mmap(file.fileno(), 0)
        pos = re.search(b'^ID', mem, re.M).start()

        while True:
            m = id_re.match(mem, pos)
            if not m:
                break

            id_type, id_value, AC = [x.decode() for x in m.groups()]
            rec = Keyword( {id_type:id_value, 'AC': AC, 'GO':[], 'HI':set(), 'WW':[], 'SY': []} )


            for m in multiline_re.finditer(mem, m.end()):
                k, v= m.groups()

                if not k: 
                    pos = m.end() # got '//' update pos for the next kw
                    break

                k = k.decode()
                v = v.decode()

                if k=='DE':
                    rec.DE = v.replace('\nDE   ', ' ')
                elif k=='SY':
                    rec.SY = [w.strip() for w in v.replace('\nSY   ', '').split(';') ]
                elif(k=='GO'):
                    rec.GO.append( v[:10] )
                elif(k=='HI'):
                    rec.HI.add( kw_split_re.split(v)[-2] ) # -1 is ID, -2 is predecessor
                elif(k=='CA'):
                    rec.CA = v
                elif(k=='WW'):
                    rec.WW.append(v)

            kw_acs[rec.AC] = rec
            kw_names[ rec.get_name() ] = rec

            
        #pprint(rec)
        #break # debug
    return kw_names, kw_acs

kw_names, kw_acs = parse_keywlist('data/IN/keywlist.txt')

Ako get_category() vrati None tj Null to je znak da je kljucna rec kategorija

### export keywrods

In [27]:
kw_df = []
kw_relations_df = []

for kw in kw_names.values():
    # Mogu da koristim dict umesto tuple ali izgubicu redosled a OrderDict je ruznije praviti od dict objekta za ovaj slucaj
    kw_entry = (kw.AC, 'keyword', kw.AC, kw.get_name(), kw.get_category(), kw.DE, kw.SY, kw.WW)
    kw_df.append(kw_entry)

    kw_relations_df.extend( (kw.AC, kw_names[p].AC, 'hi') for p in kw.HI)
    kw_relations_df.extend( (kw.AC, go, 'map_to') for go in kw.GO)

    
kw_df = pd.DataFrame(kw_df, columns=[':ID', ':LABEL', 'AC', 'name', 'category', 'def', 'synonyms', 'ww']).set_index(':ID')
kw_relations_df = pd.DataFrame(kw_relations_df, columns=[':START_ID', ':END_ID', ':TYPE']).set_index(':START_ID')

#kw_df
#kw_relations_df

# Import then export GO

In [28]:
from GO.obo_parser import  *
import json


In [29]:
with open("data/IN/go.json", "r") as obo_file:
    terms = [Term(t['id'], t) for t in json.load(obo_file)]

In [96]:
set(term_relations_df[':TYPE'])

{'consider',
 'ends_during',
 'happens_during',
 'has_part',
 'is_a',
 'negatively_regulates',
 'occurs_in',
 'part_of',
 'positively_regulates',
 'regulates',
 'replaced_by'}

In [92]:
for t in terms:
    if t.relationship:
        break

In [93]:
t.relationship

[['regulates', 'GO:0006310']]

In [97]:
term_df = []
term_relations_df = []

relation_types = ['is_a', 'replaced_by', 'consider']

for t in terms:
    # Mogu da koristim dict umesto tuple ali izgubicu redosled a OrderDict je ruznije praviti od dict objekta za ovaj slucaj
    term_entry = (t.id, 'Term', t.namespace_short(), t.id, t.name, t.namespace, t.is_obsolete, t.comment, t.definition[0], t.xr)
    term_df.append(term_entry)

    #term_relations_df.extend( (t.id, go, rt) for rt in relation_types for go in t.__dict__.get(rt) or [] )
    #term_relations_df.extend( (t.id, go, rt) for rt, go in t.relationship)

term_df = pd.DataFrame(term_df, columns=[':ID', ':LABEL', ':LABEL', 'id', 'name', 'namespace', 'is_obsolete:boolean',
                                         'comment', 'def'] ).set_index(':ID')
term_relations_df = pd.DataFrame(term_relations_df, columns=[':START_ID', ':END_ID', ':TYPE']).set_index(':START_ID')
                                         
